As a kid, I read all the time. Then, when it became a more academic endeavor, I somehow fell out of love with reading for fun. Last year I was determined to change that, and so I bought the books that came most highly recommended by my friends and started a new journey. In an effort to keep motivated, I documented what I read on Goodreads. 

When 2024 started, Goodreads prompted me to set a reading goal which felt like a fun way to motivate myself to read even more, but they don't really promote that goal in any interesting way. So I thought I'd do that for myself, by tracking my reading journey and displaying it in a more interesting way. 

So the goal is simple: Get the books I've read, calculate the approximate amount of words contained in those books, and then visualize the distance those words span. 

First, I need to get the list of books I've read in the current year from my Goodreads account.

In [2]:
from bs4 import BeautifulSoup
import requests
import time

def get_reading_list(user_id, year = "2024"):
    """
    Obtains a list of all books read in the specified year by the specified user from their Goodreads profile.
    @param user_id: The user_id, in numeric form, for the Goodreads account.
    @param year: The year to obtain the reading list for.
    """
    url = f"https://www.goodreads.com/review/list/{user_id}?date_added={year}"

    standard_headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        r = requests.get(url, headers=standard_headers)
        r.raise_for_status()  # Raises HTTPError for bad status codes
    except requests.exceptions.HTTPError as err:
        print(f"Error: {err.response.status_code}")
        time.sleep(5)
        return None
    
    # Parse the HTML and find the correct table rows to loop through for the correct data
    soup = BeautifulSoup(r.text, 'html.parser')
    books_table = soup.find("table", {"id": "books"}).find("tbody").find_all("tr")
    books_read = {}

    for book in books_table:
        item = book.find("td", {"class":"field title"}).find("div").find("a")
        # Cleaning the title of whitespace at the ends, new lines, and then white space between title & sub-title
        title = " ".join(item.text.strip().replace('\n',' ').split())
        link = item.get("href")

        books_read[title] = link

    return books_read

In [3]:
user_id = "15147556"
books_read = get_reading_list(user_id)

In [4]:
print(books_read)

{'The Kind Worth Killing (Henry Kimball/Lily Kintner, #1)': '/book/show/21936809-the-kind-worth-killing', 'Book Lovers': '/book/show/58690308-book-lovers', 'The Invisible Life of Addie LaRue': '/book/show/50623864-the-invisible-life-of-addie-larue', 'Project Hail Mary': '/book/show/54493401-project-hail-mary', 'Hell Bent (Alex Stern, #2)': '/book/show/60652997-hell-bent', 'Ninth House (Alex Stern, #1)': '/book/show/43263680-ninth-house', 'The Sword of Kaigen': '/book/show/41886271-the-sword-of-kaigen', 'The Fires of Vengeance (The Burning, #2)': '/book/show/43174603-the-fires-of-vengeance', 'The Light of All That Falls (The Licanius Trilogy, #3)': '/book/show/36111098-the-light-of-all-that-falls', 'An Echo of Things to Come (The Licanius Trilogy, #2)': '/book/show/32498052-an-echo-of-things-to-come', 'The Rage of Dragons (The Burning, #1)': '/book/show/41952489-the-rage-of-dragons', 'The Shadow of What Was Lost (The Licanius Trilogy, #1)': '/book/show/22878967-the-shadow-of-what-was-lo

Now, we have a dictionary containing the name of each book from my reading list, and the link Goodreads provided to that book's page.
Next, we can use those links to get book-level details such as the number of pages (unfortunately, word counts don't seem to be a provided data point, so that'll have to be extrapolated based on industry averages per page).

In [5]:
def fetch_book_details(book_url):
    """
    Separate function for obtaining book details (page count) from Goodreads.
    Makes two total attempts before erroring and returning None. Second attempt is delayed 5 seconds in case of limits.
    @param book_url: URL for the Goodreads book page.
    """
    standard_headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    try:
        r = requests.get(book_url, headers=standard_headers)
        r.raise_for_status()  # Raises HTTPError for bad status codes
        return r.text
    except requests.exceptions.HTTPError as err:
        print(f"Error: {err.response.status_code}")
        time.sleep(5)
        return None  # Returning None to indicate failure

In [6]:
def get_book_data(dict):
    """
    Obtains the details (page count) for books on Goodreads and compiles the total books and total pages from the dict.
    @param dict: A dictionary containing book names and links to lookup data for.
    """
    base_url = "https://www.goodreads.com"

    # Setup data structures for book processing
    book_data = []
    total_books = 0
    total_pages = 0

    # Construct book_url
    for book, link in dict.items():
        book_url = base_url + link
        retries = 2 # Number of attempts to make before erroring out

        while retries > 0:
            book_details = fetch_book_details(book_url)
            #time.sleep(1)
            if book_data is not None:
                soup = BeautifulSoup(book_details, 'html.parser')
                 # Grab pages (just the initial number)
                pages = int(soup.find("p", {"data-testid": "pagesFormat"}).text.split(" ")[0])
                # List comprehension to get a list of all genres book is tagged with
                genres = [span.find("a").find("span").text for span in soup.find_all("span", {"class": "BookPageMetadataSection__genreButton"})]
                # Construct a temp dictionary
                temp_dict = {
                    "name": book,
                    "pages": pages,
                    "genres": genres
                }
                # Append temp dictionary to our list to add to the final dict
                book_data.append(temp_dict)
                # Since we're already looping through all books here, might as well keep track for overall variables
                total_books += 1
                total_pages += pages
                break
            else:
                retries -= 1
                if retries == 0:
                    print(f"Encountered multiple errors querying book data for {book}")
    
    data_output = {
        "book_count": total_books,
        "total_pages": total_pages,
        "books": book_data
    }

    return data_output

Now, just for simplicity, I'll create a final function that just wraps all of these scraping functions into one.

In [7]:
def fetch_all(user_id, year = "2024"):
    """
    Composite operation, fetching the user's book list for the specified year and then calculating the book data for 
    every book on that list. Returns a json-like dictionary containing total books, total pages, and book details.
    @param user_id: The user_id, in numeric form, for the Goodreads account.
    @param year: The year to obtain the reading list for.
    """
    # First, get all books for reading list
    books_read = get_reading_list(user_id, year)
    # Next, calculate book data.
    books_data = get_book_data(books_read)
    return books_data

In [8]:
books_data = fetch_all(user_id)
print(books_data)

{'book_count': 20, 'total_pages': 9830, 'books': [{'name': 'The Kind Worth Killing (Henry Kimball/Lily Kintner, #1)', 'pages': 320, 'genres': ['Thriller', 'Mystery', 'Fiction', 'Mystery Thriller', 'Crime', 'Audiobook', 'Suspense']}, {'name': 'Book Lovers', 'pages': 377, 'genres': ['Romance', 'Fiction', 'Contemporary', 'Audiobook', 'Contemporary Romance', 'Adult', 'Chick Lit']}, {'name': 'The Invisible Life of Addie LaRue', 'pages': 448, 'genres': ['Fantasy', 'Fiction', 'Romance', 'Historical Fiction', 'Adult', 'Historical', 'Magical Realism']}, {'name': 'Project Hail Mary', 'pages': 476, 'genres': ['Science Fiction', 'Fiction', 'Audiobook', 'Fantasy', 'Space', 'Adult', 'Thriller']}, {'name': 'Hell Bent (Alex Stern, #2)', 'pages': 481, 'genres': ['Fantasy', 'Fiction', 'Horror', 'Mystery', 'Urban Fantasy', 'Paranormal', 'Adult']}, {'name': 'Ninth House (Alex Stern, #1)', 'pages': 461, 'genres': ['Fantasy', 'Mystery', 'Fiction', 'Horror', 'Paranormal', 'Adult', 'Urban Fantasy']}, {'name':

It takes a bit to run, and there's some built-in reattempts and slowdowns because Goodreads can be a bit fickle when scraping, but overall it's not very long and it only needs to run that one time to get our data.

For the visualization, I wanted something that works well within the realm of books that most people will read. The initial idea was to draw a distance on a map of the United States, but for people who only read a handful of books in a year, it would be an inconsequential mark. Even for those who read a lot, they wouldn't come close to crossing the entire map.

So the next idea that matched my desired parameters, and also gave a scope that was encouraging, was Mount Everest.

Somewhat surprisingly to me, Mount Everest may have a tall reputation, but compared to horizontal distances we deal with daily it's actually pretty manageable (obviously the difficulty doesn't come from pure hike length). Additionally, if we start from the Base Camp, that cuts out a good chunk. 

What if someone reads a ton though? Well, we can convert the data visualization to show a more realistic hike then complete with acclimatization steps (essentially going up, then coming back down repeatedly to get used to elevation changes).

<div>
<img src="test1.png"/ width="500">
</div>

I found this image online and modified it slightly - and will probalby modify it more for the final product - but for now it's a good starting point. I'm combining this with hiking distance data I found at https://www.alanarnette.com/everest/everestsouthroutes.php

That data is as follows:
* Base Camp to Icefall: He doesn't actually say.
* Icefall to Camp I: 1.62 miles
* Camp I to Camp II: 1.74 miles
* Camp II to Camp III: 1.64 miles
* Camp III to Camp IV: 0.8 miles
* Camp IV to Summit: 1.07 miles

The main issue is that he doesn't explicitly state a distance between Base Camp and the Icefall. Looking at lots of photos, it seems like the technical start of Khumbu Icefall on this Southern path is super close to Base Camp, so I think I can fairly safely assume the 1.62 miles stated from Icefall to Camp I is valid to use for the entire Base Camp to Camp I segment.

Additionally, the author provides a Youtube video outlining their actual path taken, which is as follows:
1. Base to Icefall 
2. Icefall back to Base
3. Base to C1 
4. C1 back to Base
5. Base to C2, 
6. C2 back to Base
7. Base to C2
8. C2 to C3
9. C3 back to C2
10. C2 back to Base
11. Base to C2
12. C2 to C3
13. C3 to C4
14. C4 to Summit

Ok, that's enough data theorizing for now. Next step is to convert the actual path on the image to a distance. Then, when the distance from the reading list is calculated, it can be converted to a spot on the trail (simplifying things and assuming linear distance on the image).

First, I'll remove the extraneous red circles just to make things easier to analyze using CV2.

<div>
<img src="test2.png"/ width="500">
</div>

In [9]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('test2.png')

# Threshold the image to isolate the trail
lower_red = np.array([0, 0, 150])
upper_red = np.array([145, 145, 255]) # Modify this to get the correct threshold for just the trail
mask = cv2.inRange(image, lower_red, upper_red)

# Can uncomment this to see the mask and verify the upper_red threshold is set correct
# cv2.imshow('Thresholded Image', mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Find contours in the thresholded image
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a dictionary to store the coordinates of the red dots that make up the trail
red_dots = {}

for i, contour in enumerate(contours):
    # Initially tried using Moments, but some of the red dots for the trail marking are too small
    # and so Moments was having issues detecting a center. Instead, can just calculate that manually.

    # Calculate the area of the contour
    area = cv2.contourArea(contour)
    if area < 0.1:
        # Assign a predefined small area value
        area = 0.1
    
    # For each contour, sum x and y values and divide by total count to get average x/y position as center
    sum_x = 0
    sum_y = 0
    count = 0

    for point in contour:
        x, y = point[0]
        sum_x += x
        sum_y += y
        count += 1

    if count != 0:
        cX = int(sum_x / count)
        cY = int(sum_y / count)
        red_dots[i + 1] = (cX, cY)

print(red_dots)

{1: (404, 747), 2: (408, 744), 3: (413, 741), 4: (418, 738), 5: (422, 736), 6: (428, 735), 7: (432, 732), 8: (428, 728), 9: (425, 724), 10: (421, 720), 11: (417, 717), 12: (413, 712), 13: (410, 708), 14: (409, 703), 15: (411, 699), 16: (416, 696), 17: (427, 695), 18: (421, 695), 19: (443, 694), 20: (438, 694), 21: (432, 694), 22: (448, 693), 23: (454, 692), 24: (465, 691), 25: (470, 690), 26: (481, 689), 27: (486, 688), 28: (502, 686), 29: (497, 686), 30: (507, 685), 31: (513, 683), 32: (518, 682), 33: (523, 680), 34: (528, 677), 35: (532, 673), 36: (535, 669), 37: (535, 664), 38: (533, 659), 39: (529, 654), 40: (525, 651), 41: (521, 647), 42: (517, 644), 43: (513, 640), 44: (509, 637), 45: (505, 634), 46: (500, 630), 47: (497, 627), 48: (496, 621), 49: (500, 618), 50: (505, 615), 51: (510, 614), 52: (515, 613), 53: (520, 612), 54: (526, 611), 55: (531, 609), 56: (536, 608), 57: (541, 606), 58: (546, 604), 59: (551, 601), 60: (556, 599), 61: (561, 596), 62: (566, 595), 63: (601, 582), 

This can easily be checked just by opening the image in Paint and verifying the (x,y) coordinates of the red dots after zooming in. Everything looks good. 

The next step is to get the euclidean distance between these (x,y) coordinates. This will give the sum total distance for the entire path going from point-to-point for conversion into mileage.

In [10]:
import math

def calculate_distance(point1, point2):
    # Simple function to calculate euclidean distance
    x1, y1 = point1
    x2, y2 = point2
    return round(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2), 2)

In [11]:
# First point will be set to 0 unless I want to change it to the base camp duration later on, so this exempts it
previous_point = None

# Creating a new dict to modify
trail = dict(red_dots)

for key, coordinates in trail.items():
    if previous_point is not None:
        distance = calculate_distance(previous_point, coordinates)
        trail[key] = [coordinates, distance]
    else:
        trail[key] = [coordinates, 0]
    previous_point = coordinates

# Calculating the total trail distance
total_distance = 0
for data in trail.values():
    _, distance = data
    total_distance += distance

print(trail)
print(round(total_distance,2))

{1: [(404, 747), 0], 2: [(408, 744), 5.0], 3: [(413, 741), 5.83], 4: [(418, 738), 5.83], 5: [(422, 736), 4.47], 6: [(428, 735), 6.08], 7: [(432, 732), 5.0], 8: [(428, 728), 5.66], 9: [(425, 724), 5.0], 10: [(421, 720), 5.66], 11: [(417, 717), 5.0], 12: [(413, 712), 6.4], 13: [(410, 708), 5.0], 14: [(409, 703), 5.1], 15: [(411, 699), 4.47], 16: [(416, 696), 5.83], 17: [(427, 695), 11.05], 18: [(421, 695), 6.0], 19: [(443, 694), 22.02], 20: [(438, 694), 5.0], 21: [(432, 694), 6.0], 22: [(448, 693), 16.03], 23: [(454, 692), 6.08], 24: [(465, 691), 11.05], 25: [(470, 690), 5.1], 26: [(481, 689), 11.05], 27: [(486, 688), 5.1], 28: [(502, 686), 16.12], 29: [(497, 686), 5.0], 30: [(507, 685), 10.05], 31: [(513, 683), 6.32], 32: [(518, 682), 5.1], 33: [(523, 680), 5.39], 34: [(528, 677), 5.83], 35: [(532, 673), 5.66], 36: [(535, 669), 5.0], 37: [(535, 664), 5.0], 38: [(533, 659), 5.39], 39: [(529, 654), 6.4], 40: [(525, 651), 5.0], 41: [(521, 647), 5.66], 42: [(517, 644), 5.0], 43: [(513, 640)

So with the total distance, that can now be mapped to the mileage of our hike from before. Therefore

1179.31 units = (1.62 + 1.74 + 1.64 + 0.8 + 1.07 miles) = 6.87 miles

So, if our calculated reading length was 3.1 miles, it's as simple as doing the following calculation:

(3.1 / 6.87) * 1179.31 = 532.15 units hiked

Then we can just loop through the trail until we've hiked 532.15 units, and that will be how far we've read to. We could go one point prior, or find the average distance between the point before 532.15 units and the point after that is the correct distance, but I think that's a level of specificity that's unnecessary.

In [12]:
target_distance = 535.15
accumulated_distance = 0
target_coordinates = None

for key, data in trail.items():
    coordinates, distance = data
    accumulated_distance += distance
    if accumulated_distance >= target_distance:
        target_coordinates = coordinates
        break

print(f"Target: {target_coordinates}")

Target: (551, 487)


To quickly verify, since this should be just below 50% of the way up the trail, we can draw a dot there to see.

In [13]:
from PIL import Image, ImageDraw

image = Image.open("test2.png")
draw = ImageDraw.Draw(image)

x,y = target_coordinates
left = x-5
top = y-5
right = x+5
bottom = y+5

draw.rectangle([left, top, right, bottom], fill = "red")
image.show()

It's a bit hard to tell if that's close to 50% of the way, but it looks fairly correct since the start contains much more twists in the trail than the latter part of the hike where it's mostly straightforward vertical climbing. 

Ok, now we just need a quick calculation for the total distance from our book data. 

In [14]:
print(books_data['total_pages'])

9830


The industry standard for words per page seems to be ~300. I tested with a few random books from my shelf and this definitely seemed accurate for paperbacks, but a bit low for larger hardcovers. However, Goodreads data seems to tend towards a larger number of pages (for *The Will of the Many* it was +25 pages compared to my hardcover) since it commonly uses Kindle editions.

Even for my paperback copy of *The Stone Sky*, which is the edition Goodreads supposedly uses, it was +20 pages off, possibly due to counting appendecies and such or perhaps it's just a different paperback print format. 

Since 300 words/page seems like it'll be a slight underestimation (my random page count for *The Stone Sky* was 284, and for *The Will of the Many* was a whopping 500), I think it'll end up working nicely in conjunction with Goodreads' inflated values. 

So, 300 words/page. In English, the average word length is 4.7 characters. Both books I've been using for physical data have a 5-letter word come out to almost exactly 1/4 of an inch. That means the average word, at 4.7 characters, would be ~0.235 inches. 

In [15]:
def get_reading_distance(total_pages, wpp = 300, ipw = 0.235):
    """
    Returns the reading distance for a total number of pages as a mileage.
    @param total_pages: A numeric value of the total number of pages.
    @param wpp: The average words per page, default is 300.
    @param ipw: The average inches per word, default is 0.235.
    """
    # 12 inches/ft, 5280 ft/mile, 63360 inches/mile
    books_distance = (total_pages * wpp * ipw) / 63360
    return books_distance

books_distance = get_reading_distance(books_data['total_pages'])
print(books_distance)

10.937736742424242


So my 2024 reading list is already more than the distance, so I would have either completed my journey or I could choose the actual climbing route which loops back to the base camp after hitting certain elevations. No matter, because we can make both an option. 

Might as well check the data for my 2023 reading list which may be shorter as another testing point.

In [16]:
test_2023_reading = fetch_all(user_id, "2023")
test_2023_distance = get_reading_distance(test_2023_reading['total_pages'])
print(test_2023_distance)

9.812807765151515


In [17]:
def draw_trail(trail, camp_index):
    # Draw a line on the trail up until a specified index
    image = Image.open('everest_template_clean.png')
    draw = ImageDraw.Draw(image)
    color = "blue"
    thickness = 5

    for i in range(1, camp_index + 1):
        if i in trail:
            x, y = trail[i][0]
            if i > 1:
                prev_x, prev_y = trail[i - 1][0]
                draw.line([(prev_x, prev_y), (x, y)], fill = color, width = thickness, joint = 'curve')

    image.save('everest_reading_journey.png')

In [49]:
def get_label_data(distance, acclimatization = False):
    """
    Returns a string based on the distance provided explaining the percentage of the journey completed, as well as what 
    major landmarks have been passed. 
    """

    distance = round(distance, 2)
    if acclimatization:
        percent_hiked = round((distance / 30.07) * 100, 2)
        label = [f"Congratulations, you have read for a total of {distance} miles, ",
                  f"which is {percent_hiked}% of the total journey!"]
    else:
        percent_hiked = round((distance / 6.87) * 100, 2)
        label = [f"Congratulations, you have read for a total of {distance} miles, ", 
                 f"which is {percent_hiked}% of the total journey!"]

    return label

In [50]:
from PIL import ImageFont

def draw_label(camp_index, trail, acclimatization = False, distance = 0):
    image = Image.open('everest_reading_journey.png')
    draw = ImageDraw.Draw(image)
    # Cut-off for when we should swap the label line from down to up.
    midpoint_y = image.height/2
    
    color = "red"
    thickness = 5

    original_distance = distance

    if acclimatization:
        if distance <= 0:
            print("Please provide a valid distance if using acclimatization.")
            return
        
        # Determine whether we're moving forward or not based on step
        acclim_directions = [1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1]
        acclim_distances = [1.62, 1.62, 1.62, 1.62, 3.36, 3.36, 3.36, 1.64, 1.64, 3.36, 3.36, 1.64, 0.8, 1.07]

        if camp_index == len(trail):
            # Pre-check if we've reached the end of the trail even with acclimatization to make things easier
            net_distance = sum(acclim_distances)
            camp_coordinates = trail[max(trail.keys())][0]
        else:   
            net_distance = 0

            for i in range(len(acclim_distances)):
                # Check if this is the final step
                if distance < acclim_distances[i]:
                    # If so, add/substract the remaining distance to our net distance
                    net_distance += distance * acclim_directions[i]
                    break
                
                # Remove distance and add/remove from net distance based on step direction
                distance -= acclim_distances[i]
                net_distance += acclim_distances[i] * acclim_directions[i]
            
            accumulated_distance = 0
            # Trail distance is in units, parameter distance is in miles, so we convert
            trail_length = 0
            for data in trail.values():
                _, length = data
                trail_length += length
            net_distance = net_distance * (trail_length / 6.87)
            
            for key, data in trail.items():
                coordinates, length = data
                accumulated_distance += length
                if accumulated_distance >= net_distance:
                    camp_coordinates = coordinates
                    break
        
    else:
        camp_coordinates = trail[camp_index][0]
        
    x,y = camp_coordinates

    if y < midpoint_y:
        # This means we're in the top-half of the image, so we angle the line down
        x_new = x - 50
        y_new = y + 50
    else:
        x_new = x - 50
        y_new = y - 50
    
    draw.line([(x_new, y_new), (x, y)], fill = color, width = thickness, joint = 'curve')
    draw.line([(x_new - 100, y_new), (x_new, y_new)], fill = color, width = thickness, joint = 'curve')

    font_size = 16
    font_color = "black"
    font = ImageFont.truetype("Roboto-Regular.ttf", font_size)

    label = get_label_data(original_distance, acclimatization)
    for line in label:
        draw.text((50, y_new - 50), line, fill = font_color, font = font)
        y_new += 25
    image.show()
    image.save('everest_reading_journey.png')

In [20]:
def plot_journey_acclim(distance, trail):
    # Distance for individual steps in acclimatization climb
    acclim_distances = [1.62, 1.62, 1.62, 1.62, 3.36, 3.36, 3.36, 1.64, 1.64, 3.36, 3.36, 1.64, 0.8, 1.07]
    # 0-3: up to C1, 4-6: up to C2, 7-11: up to C3, 12: up to C4, 13: up to Summit
    
    for i in range(len(acclim_distances)):
        # Find the step in the climb based on the distance
        if distance < sum(acclim_distances[:i+1]):
            acclim_journey_step = i
            break
        else:
            acclim_journey_step = 999

    # Set the camp_index to the index of highest point reached in the trail
    if acclim_journey_step < 4:
        camp_index = 62
    elif acclim_journey_step < 7:
        camp_index = 83
    elif acclim_journey_step < 12:
        camp_index = 110
    elif acclim_journey_step < 13:
        camp_index = 145
    else:
        camp_index = len(trail)
    
    # Draw the hiking path up to the highest point
    draw_trail(trail, camp_index)
    draw_label(camp_index, trail, True, distance)


In [35]:
def plot_journey_single(distance, trail):
    accumulated_distance = 0
    target_coordinates = None

    # Trail distance is in units, parameter distance is in miles, so we convert
    original_distance = distance
    trail_length = 0
    for data in trail.values():
        _, length = data
        trail_length += length
    distance = distance * (trail_length / 6.87)

    # If distance surpasses trail length, set index to max for summit
    if distance >= trail_length:
        camp_index = len(trail)
    # Otherwise find index of nearest point on trail to our distance
    else:
        for key, data in trail.items():
            coordinates, length = data
            accumulated_distance += length
            if accumulated_distance >= distance:
                camp_index = key
                break
    
    draw_trail(trail, camp_index)
    draw_label(camp_index, trail, False, original_distance)

In [22]:
def plot_journey(distance, trail, acclimatization = False):
    if acclimatization is not True:
        plot_journey_single(distance, trail)
    else:
        plot_journey_acclim(distance, trail)

Alright, so all of the functions are done. In total, we have:

plot_journey -> This plots the entire journey based on the distance and the trail, and determines whether to use a single-trip setting or the acclimatization setting. It will call the corresponding function based on that parameter.

plot_trail & plot_label -> These will plot the trail, up to the highest point for single journey, or the highest camp reached for acclimatization journies, and then also add a label at the end-point for each journey describing the percentage completed, etc.

We have also used earlier functions to get our distance in books_distance, our trail data in trail, and any other stored data we may need in books_data. 

I also quickly edited the image and changed the filepath retroactively to point towards the new, cleaned-up version of Everest where all the campsites are marked on the right to make label-drawing more consistent on the left-side without worrying about overlaps.

Let's see how it looks put together.

In [27]:
print(books_distance)

10.937736742424242


In [51]:
plot_journey(books_distance, trail, True)